In [ ]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.tuner import Tuner
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from  pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.metrics.quantile import QuantileLoss
import pandas as pd
import torchvision


In [ ]:
df = pd.read_csv('cex4WindDataInterpolated.csv')

# convert time to datetime
df['t_new'] = pd.to_datetime(df['t'])

# time collumns: t, toy

# target must be float
# df['p'] = df['p'].astype(float)

# variables to use for predictions: Ws, Wd, T

# drop NA
# df.dropna(inplace=True)

# add forward fill
# df.ffill(inplace=True)

df.head()

In [ ]:
nan_count = df.isna().sum()
nan_count

In [ ]:
df = df.drop(['Ws2', 'Wd2', 'T2', 'Ws3', 'Wd3', 'T3'], axis=1)
df.head()

In [ ]:
max_encoder_length = 36
max_prediction_length = 1 # old: 6
training_cutoff = "2001-01-01"

df.ffill(inplace=True)
df['group_id'] = 'wind_farm_1'  # All rows will have this same value

# min_date = df['t_new'].min()
# df["time_idx"] = df["t_new"].map(lambda current_date: (current_date - min_date).days)

datetime_to_index = {date: idx for idx, date in enumerate(df['t_new'].unique())}
df['time_idx'] = df['t_new'].map(datetime_to_index)

In [ ]:
df = df.drop(['t', 'toy', 't_new'], axis=1)

In [ ]:
df = df.drop(['Ws1', 'Wd1', 'T1'], axis=1)

In [ ]:
df

In [ ]:
training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= 1000],
    time_idx="time_idx",
    target="p",
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=[
        "p" #, "Ws1", "Wd1", "T1"
    ],
    allow_missing_timesteps=True
)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.models import TemporalFusionTransformer
# stop training, when loss metric does not improve on validation set

validation = TimeSeriesDataSet.from_dataset(training, df, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 16
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=1)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=1)

lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # log to tensorboard
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
# create trainer
trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,  # train on CPU, use gpus = [0] to run on GPU
    gradient_clip_val=0.1,
    limit_train_batches=30,  # running validation every 30 batches
    # fast_dev_run=True,  # comment in to quickly check for bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)
# initialise model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # biggest influence network size
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # QuantileLoss has 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # log example every 10 batches
    reduce_on_plateau_patience=4,  # reduce learning automatically
)
tft.size() # 29.6k parameters in model
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
# Same Issue
# https://github.com/jdb78/pytorch-forecasting/issues/376

# create validation and training dataset
validation = TimeSeriesDataSet.from_dataset(training, df, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 16
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)

# define trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=1, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
)

# create the model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=2,
    reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

# find optimal learning rate (set limit_train_batches to 1.0 and log_interval = -1)
res = Tuner(trainer).lr_find(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader, early_stop_threshold=1000.0, max_lr=0.3,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

# fit the model
trainer.fit(
    tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader,
)